In [1]:
pip install biopython python-docx

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from Bio import SeqIO
from docx import Document
from docx.shared import Pt
from docx.oxml.text.run import CT_R

# 1. Configurar rutas
base_path = r'D:\ExamenParcial2ComputacionParaBioingenieria\parcial2_Machicado_Martinez_GabrielAntonio_ejercicio1'
genomas_path = os.path.join(base_path, 'genomas_ecoli')
genes_path = os.path.join(base_path, 'genes_resistentes')
plantilla_path = os.path.join(base_path, 'informe_bacteria_XXXX.docx')
output_path = os.path.join(base_path, 'informes_generados')

os.makedirs(output_path, exist_ok=True)

# 2. Cargar genes resistentes
genes_resistentes = {}
for gen_file in ['blatem1.fasta', 'blactxm15.fasta']:
    gene_id = gen_file.split('.')[0]
    gene_record = next(SeqIO.parse(os.path.join(genes_path, gen_file), 'fasta'))
    gene_seq = str(gene_record.seq).upper()
    genes_resistentes[gene_id] = gene_seq
    print(f"Gen {gene_id} cargado con {len(gene_seq)} bases. Ejemplo: {gene_seq[:30]}...")

# 3. Buscar genes en genomas
resultados = []

for archivo_genoma in os.listdir(genomas_path):
    if archivo_genoma.endswith('.fna'):
        ruta = os.path.join(genomas_path, archivo_genoma)
        print(f"\nAnalizando: {archivo_genoma}")
        secuencia = ''
        for record in SeqIO.parse(ruta, 'fasta'):
            secuencia += str(record.seq)

        secuencia = secuencia.upper()
        print(f"Longitud total del genoma: {len(secuencia)}")

        hallazgos = []
        for gen, secuencia_gen in genes_resistentes.items():
            if secuencia_gen in secuencia:
                print(f">>> ¡Gen {gen} encontrado en {archivo_genoma}!")
                hallazgos.append(gen)

        resultados.append({
            'archivo': archivo_genoma,
            'hallazgos': hallazgos
        })

# 4. Reemplazo robusto
def reemplazar_en_parrafo(parrafo, reemplazos, nombre_bacteria):
    texto = ''.join(run.text for run in parrafo.runs)
    texto_original = texto
    modificado = False

    for buscar, reemplazar in reemplazos.items():
        if buscar.lower() in texto.lower():
            texto = texto.lower().replace(buscar.lower(), reemplazar)
            modificado = True

    if modificado:
        # Tratamiento especial para REF.:
        if texto_original.strip().lower().startswith("ref.:"):
            # Borrar todos los runs
            for run in parrafo.runs:
                run._element.getparent().remove(run._element)

            # Reconstruir: separar REF + resto hasta la bacteria + bacteria + lo demás
            before_bacteria = texto.split(nombre_bacteria)[0].upper()
            after_bacteria = texto.split(nombre_bacteria)[1].upper()

            run1 = parrafo.add_run(before_bacteria)
            run1.font.name = 'Tahoma'
            run1.bold = True
            run1.font.size = Pt(10)

            run2 = parrafo.add_run(nombre_bacteria)
            run2.font.name = 'Tahoma'
            run2.bold = False
            run2.font.size = Pt(10)

            run3 = parrafo.add_run(after_bacteria)
            run3.font.name = 'Tahoma'
            run3.bold = True
            run3.font.size = Pt(10)
        else:
            # Borrar y reemplazar sin formato especial
            for run in parrafo.runs:
                run._element.getparent().remove(run._element)
            parrafo.add_run(texto)

# 5. Generar informes
for r in resultados:
    nombre_bacteria = r['archivo'].replace('.fna', '')
    estado_resistencia = "POSITIVO" if r['hallazgos'] else "NEGATIVO"
    informe_nuevo = os.path.join(output_path, f'informe_bacteria_{nombre_bacteria}.docx')
    
    doc = Document(plantilla_path)

    for p in doc.paragraphs:
        reemplazar_en_parrafo(p, {
            "nombre de la bacteria aquí!!!": nombre_bacteria,
            "POSITIVO/NEGATIVO": estado_resistencia
        }, nombre_bacteria)

    doc.save(informe_nuevo)
    print(f"✔ Informe generado para {nombre_bacteria}")

print(f"\n✅ Se generaron {len(resultados)} informes en: {output_path}")

Gen blatem1 cargado con 979 bases. Ejemplo: TCGGGGAAATGTGCGCGGAACCCCTATTTG...
Gen blactxm15 cargado con 2315 bases. Ejemplo: TGGGATTTTTGATTTTATTGAAAATGACCT...

Analizando: GCF_050389795.1_MS10495_ARL02201_ST410_genomic.fna
Longitud total del genoma: 3106818

Analizando: GCF_050516345.1_ASM5051634v1_genomic.fna
Longitud total del genoma: 5177992

Analizando: GCF_050516355.1_ASM5051635v1_genomic.fna
Longitud total del genoma: 4142449

Analizando: GCF_050516365.1_ASM5051636v1_genomic.fna
Longitud total del genoma: 5039403

Analizando: GCF_050516375.1_ASM5051637v1_genomic.fna
Longitud total del genoma: 4142451

Analizando: GCF_050516385.1_ASM5051638v1_genomic.fna
Longitud total del genoma: 4142452

Analizando: GCF_050563505.1_ASM5056350v1_genomic.fna
Longitud total del genoma: 4142445

Analizando: GCF_050568845.1_2848_genomic.fna
Longitud total del genoma: 5337260
✔ Informe generado para GCF_050389795.1_MS10495_ARL02201_ST410_genomic
✔ Informe generado para GCF_050516345.1_ASM5051634v1_gen